In [6]:
from elasticsearch import Elasticsearch


In [7]:
es_client = Elasticsearch('http://localhost:9200') 


In [8]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
from tqdm.auto import tqdm


/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:

for doc in documents:
    es_client.index(index=index_name, document=doc)

In [12]:
query = "How do I execute a command in a running docker container?"

In [36]:

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    res = ""
    
    for hit in response['hits']['hits']:
        context_template = """
        Q: {question}
        A: {text}
        """.strip()
        
        context = context_template.format(question=hit["_source"]["question"], text=hit["_source"]["text"])
        res += context + "\n\n"
    
    return res

In [37]:
def query_to_prompt(query):
    search_results = elastic_search(query)

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    prompt = prompt_template.format(question=query, context=search_results)
    return prompt

In [39]:
print(len(query_to_prompt(query)))

1512


In [40]:
import tiktoken

In [41]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [42]:
print(encoding.encode(query_to_prompt(query)))

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 271, 7649, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 558, 1944, 271, 150339, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 3901, 1944, 271, 31810, 8099, 734, 271, 1486, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 309, 355, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 309, 355, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 

In [43]:
print(len(encoding.encode(query_to_prompt(query))))

332
